In [123]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from collections import Counter
from itertools import combinations
warnings.filterwarnings('ignore')

In [54]:
dataset = pd.read_csv('../data/heart_disease_dataset_UCI.csv')
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [55]:
standardScalar = StandardScaler()
target_columns = ['age','trestbps','chol','thalach','oldpeak']
dataset[target_columns] = standardScalar.fit_transform(dataset[target_columns])
dataset.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0.952197,1,3,0.763956,-0.256334,1,0,0.015443,0,1.087338,0,0,1,1
1,-1.915313,1,2,-0.092738,0.072199,0,1,1.633471,0,2.122573,0,0,2,1
2,-1.474158,0,1,-0.092738,-0.816773,0,0,0.977514,0,0.310912,2,0,2,1
3,0.180175,1,1,-0.663867,-0.198357,0,1,1.239897,0,-0.206705,2,0,2,1
4,0.290464,0,0,-0.663867,2.082050,0,1,0.583939,1,-0.379244,2,0,2,1


In [202]:
#Dividing into input and output
X= dataset.drop(['target'], axis=1)
Y= dataset['target']
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size=0.25, random_state=40)

In [201]:
def dist(x, y, p):
    dim = len(x)
    dist = 0
    for i in range(dim):
        dist += abs(x[i] - y[i])**p
    return dist**(1/p)

In [210]:
# Calculate the minkowski distance between every new data point in X_test and all points in X_train, show accuracy
def knn (distfunction, X_train_arr, X_test_arr, k):
    Y_prediction = []
    for i in X_test_arr:
        distances = []
        for j in X_train_arr:
            distances.append(dist(i,j,distfunction))
        df_dist = pd.DataFrame(index=Y_train.index, columns=['dist'], data=distances)
        df_neighbors = df_dist.sort_values(by=['dist'], axis=0)[:k]
        neighbortally = Counter(Y_train[df_neighbors.index])
        Y_prediction.append(neighbortally.most_common(1)[0][0])

    #cmknn = confusion_matrix(Y_test, Y_prediction)
    #print("No Dim Reduce |", distfunc, "| K Neighbors =", k,'\n', cmknn)
    #print(Y_prediction)
    #print("\nAccuracy=", accuracy_score(Y_test, Y_prediction),"\n")
    return accuracy_score(Y_test, Y_prediction)

In [211]:
X_train_arr = X_train.to_numpy()
X_test_arr = X_test.to_numpy()
k = 5
distfunc = 1 # P value for minkowski distance
#[manhattan = 1, euclidean = 2]

In [212]:
#SFS

sfsmax = 5
selectfeat = []
tempselect = []
features = []
traindataselect = []
testdataselect = []
bestaccuracy = 0
bestfeat = ''
features = list(X_train.columns)
#print(features)

while(len(selectfeat) < sfsmax):
    for j in range(len(selectfeat)):
            tempselect.append(selectfeat[j])
    for i in range(len(features)):
        tempselect.append(features[i])
        print('Featureset: {}'.format(tempselect))
        traindataselect = X_train[tempselect]
        testdataselect = X_test[tempselect] 
        
        accuracy = knn(distfunc, traindataselect.to_numpy(), testdataselect.to_numpy(), k)
        print('Accuracy: {}'.format(accuracy))

        if (bestaccuracy < accuracy):
            bestaccuracy = accuracy
            bestfeat = features[i]
        tempselect.pop(-1)

    selectfeat.append(bestfeat)
    features.remove(bestfeat)
    print("============================================")
    print("\nBest feature to add: {}".format(bestfeat))
    print("Accuracy with feature addition: {}".format(bestaccuracy))
    print("Total features added: {}\n".format(len(selectfeat)))
    print("============================================")
    bestaccuracy = 0
    del tempselect[:]


print("Final featureset: {}".format(selectfeat))
traindataselect = X_train[selectfeat]
testdataselect = X_test[selectfeat] 
accuracy = knn(distfunc, traindataselect.to_numpy(), testdataselect.to_numpy(), k)

Featureset: ['age']
Accuracy: 0.5526315789473685
Featureset: ['sex']
Accuracy: 0.5526315789473685
Featureset: ['cp']
Accuracy: 0.7763157894736842
Featureset: ['trestbps']
Accuracy: 0.5394736842105263
Featureset: ['chol']
Accuracy: 0.5394736842105263
Featureset: ['fbs']
Accuracy: 0.5526315789473685
Featureset: ['restecg']
Accuracy: 0.6052631578947368
Featureset: ['thalach']
Accuracy: 0.6710526315789473
Featureset: ['exang']
Accuracy: 0.7763157894736842
Featureset: ['oldpeak']
Accuracy: 0.6842105263157895
Featureset: ['slope']
Accuracy: 0.5526315789473685
Featureset: ['ca']
Accuracy: 0.7631578947368421
Featureset: ['thal']
Accuracy: 0.7894736842105263

Best feature to add: thal
Accuracy with feature addition: 0.7894736842105263
Total features added: 1

Featureset: ['thal', 'age']
Accuracy: 0.7236842105263158
Featureset: ['thal', 'sex']
Accuracy: 0.7763157894736842
Featureset: ['thal', 'cp']
Accuracy: 0.7763157894736842
Featureset: ['thal', 'trestbps']
Accuracy: 0.7763157894736842
Feature